In [1]:
import pandas as pd
import numpy as np
import openpyxl


In [18]:
df_liz = pd.read_excel(
    "../data/DataLizen.xlsx",
    sheet_name="Lizenzierte",
    header=2
)
df_liz.head()

,Vereinsname,Geschlecht Person,Altersklasse,Anz. Spieler/innen in Vereinen,Person-Nr anonymisiert
0,FC Adliswil (11001),M,Jun.B,1,8243
1,FC Adliswil (11001),M,Jun.B,1,8318
2,FC Adliswil (11001),W,Jun.C,1,8791
3,FC Adliswil (11001),M,Jun.C,1,10572
4,FC Adliswil (11001),M,Jun.B,1,11055


In [19]:
df_spiel = pd.read_excel(
    "../data/DataLizen.xlsx",
    sheet_name="Spiele",
    header=2
)
df_spiel.head()

,Vereinsname,Teamname,Liga,Begegnung,Position,Anz. Einsätze Spieler/innen,Total Einsatz-Minuten,Anz. Tore,Person-Nr anonymisiert
0,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld linksinnen,1.0,67,NaN,230707
1,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld,NaN,0,NaN,235805
2,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld rechtsinnen,1.0,90,NaN,347144
3,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Mittelfeld linksinnen,1.0,1,NaN,440159
4,GC Frauenfussball (11705),GC Frauenfussball (AWSL),Women’s Super League,01.11.2025 FC Aarau Frauen - GC Frauenfussball,Verteidigung linksaussen,1.0,90,NaN,541906


In [8]:
df_liz_CD = df_liz[(df_liz["Altersklasse"]=="Jun.C") | (df_liz["Altersklasse"]=="Jun.D")]
df_liz_CD

,Vereinsname,Geschlecht Person,Altersklasse,Anz. Spieler/innen in Vereinen,Person-Nr anonymisiert
2,FC Adliswil (11001),W,Jun.C,1,8791
3,FC Adliswil (11001),M,Jun.C,1,10572
5,FC Adliswil (11001),M,Jun.D,1,17699
6,FC Adliswil (11001),M,Jun.C,1,22391
24,FC Adliswil (11001),W,Jun.D,1,67719
...,...,...,...,...,...
64816,Zürich City SC (11560),W,Jun.D,1,1397388
64818,Zürich City SC (11560),W,Jun.C,1,1398026
64819,Zürich City SC (11560),W,Jun.D,1,1401797
64820,Zürich City SC (11560),W,Jun.C,1,1401965


In [9]:
mask = df_spiel["Person-Nr anonymisiert"].isin(df_liz_CD["Person-Nr anonymisiert"]).copy()
df_games = df_spiel[mask].copy()
df_games.info()

df_games = pd.merge(df_liz_CD[["Person-Nr anonymisiert", "Geschlecht Person"]], df_games, on="Person-Nr anonymisiert", how="right")
df_games.info()



<class 'pandas.core.frame.DataFrame'>
Index: 35267 entries, 366 to 127254
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Vereinsname                  35267 non-null  object 
 1   Teamname                     35267 non-null  object 
 2   Liga                         35267 non-null  object 
 3   Begegnung                    35267 non-null  object 
 4   Position                     35267 non-null  object 
 5   Anz. Einsätze Spieler/innen  34691 non-null  float64
 6   Total Einsatz-Minuten        35267 non-null  int64  
 7   Anz. Tore                    5162 non-null   float64
 8   Person-Nr anonymisiert       35267 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 2.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35267 entries, 0 to 35266
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                

In [10]:
# the NaN goals, it is 0 goals, must be all integers
df_games["Anz. Tore"] = (
    pd.to_numeric(df_games["Anz. Tore"], errors="coerce")
      .fillna(0)
      .astype(int)
)

# list of best players
players_list = (
    df_games.groupby("Person-Nr anonymisiert", as_index=False).agg({
        "Anz. Tore": "sum",
        "Total Einsatz-Minuten": "sum",
        "Liga": lambda x: ", ".join(sorted(set(x.dropna()))),
        "Position": lambda x: ", ".join(sorted(set(x.dropna()))), 
        "Vereinsname": "last", 
        "Geschlecht Person": "first"
        })
        .rename(columns={"Anz. Tore": "Goals", "Person-Nr anonymisiert" : "Id-Player", "Total Einsatz-Minuten":"Minutes", "Geschlecht Person":"Sex"})
        .sort_values("Goals", ascending=False)
)
print(players_list.shape)
players_list.head()

(5504, 7)


,Id-Player,Goals,Minutes,Liga,Position,Vereinsname,Sex
3477,954309,51,810,"Juniorinnen B, Juniorinnen C","Flügel links, Sturmspitze links, Sturmspitze r...",FC Embrach (11266),W
1902,539951,30,786,"Junioren B 1. Stärkeklasse, Junioren C 1. Stär...","Flügel links, Mittelfeld, Mittelfeld linksauss...",FC Dielsdorf (11008),M
1466,416448,27,1186,"Junioren A Promotion, Junioren C 1. Stärkeklasse","Mittelsturm, Sturmspitze, Tor",FC Mönchaltorf (11035),M
3047,839380,26,723,"Junioren B 1. Stärkeklasse, Junioren C 2. Stär...","Mittelfeld linksaussen, Sturmspitze, Sturmspit...",FC Srbija ZH (11516),M
4554,1269933,26,785,Junioren C 2. Stärkeklasse,"Flügel links, Sturmspitze links",FC Mönchaltorf (11035),M


In [11]:
club_top = (
    players_list
    .groupby("Vereinsname", as_index=False)
    .agg(
        Cant_Player=("Id-Player", "count"),
        Cant_Goal=("Goals", "sum")
    )
    .sort_values("Cant_Goal", ascending=False)
)
club_top

,Vereinsname,Cant_Player,Cant_Goal
17,FC Dielsdorf (11008),66,218
36,FC Herrliberg (11057),89,208
93,FC Witikon (11496),125,203
24,FC Embrach (11266),90,186
9,FC Birmensdorf (11004),64,183
...,...,...,...
112,US Virtus Badolato (11559),19,7
53,FC Neunkirch (11109),2,5
99,FFC Lionesses (11456),1,3
79,FC Thayngen (11108),6,2


In [17]:
print(players_list.columns.to_list())
print(club_top.columns.to_list())

['Id-Player', 'Goals', 'Minutes', 'Liga', 'Position', 'Vereinsname', 'Sex']
['Vereinsname', 'Cant_Player', 'Cant_Goal']


In [12]:
club_top.to_csv("../data/club_top.csv", index=False, encoding="utf-8") 

players_list.to_csv("../data/players_list.csv", index=False, encoding="utf-8") 


In [10]:
club_top2 = pd.read_csv("../data/club_top.csv", encoding="utf-8")
club_top = club_top2.copy()
club_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Vereinsname  113 non-null    object
 1   Cant_Player  113 non-null    int64 
 2   Cant_Goal    113 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.8+ KB


In [8]:
players_list2 = pd.read_csv("../data/players_list.csv", encoding="utf-8")
players_list =players_list2.copy()

In [11]:
players_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5504 entries, 0 to 5503
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id-Player    5504 non-null   int64 
 1   Goals        5504 non-null   int64 
 2   Minutes      5504 non-null   int64 
 3   Liga         5504 non-null   object
 4   Position     5504 non-null   object
 5   Vereinsname  5504 non-null   object
 6   Sex          5504 non-null   object
dtypes: int64(3), object(4)
memory usage: 301.1+ KB


In [17]:
Verein ="FC Dielsdorf (11008)"
df_filt = players_list[players_list["Vereinsname"]==Verein]
mask =df_filt["Goals"].max()
best_player = df_filt[df_filt["Goals"] == mask]
print(best_player)

print(best_player["Id-Player"])

   Id-Player  Goals  Minutes  \
1     539951     30      786   

                                                Liga  \
1  Junioren B 1. Stärkeklasse, Junioren C 1. Stär...   

                                            Position           Vereinsname Sex  
1  Flügel links, Mittelfeld, Mittelfeld linksauss...  FC Dielsdorf (11008)   M  
1    539951
Name: Id-Player, dtype: int64


In [31]:
list_position = (players_list2["Position"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
print(list_position)

['Flügel links' 'Sturmspitze links' 'Sturmspitze rechts' 'Mittelfeld'
 'Mittelfeld linksaussen' 'Mittelsturm' 'Sturmspitze' 'Tor' 'Ersatz (S)'
 'Flügel rechts' 'Mittelfeld defensiv' 'Mittelfeld defensiv links'
 'Mittelfeld rechtsinnen' 'hängende Spitze' 'Mittelfeld rechts'
 'Innenverteidigung links' 'Libero' 'Verteidigung mitte'
 'hängende Spitze links' 'Hängende Spitze rechts' 'Mittelfeld linksinnen'
 'Mittelfeld links' 'Verteidigung rechts' 'Innenverteidigung rechts'
 'Mittelfeld rechtsaussen' 'Verteidigung linksaussen' 'Verteidigung links'
 'Verteidigung vorne' 'Mittelfeld defensiv rechts'
 'Verteidigung rechtsaussen' '?' 'Ersatz (T)']


In [33]:
list_Liga = (players_list2["Liga"]
                 .str.split(",")
                 .explode()
                 .str.strip()
                 .dropna()
                 .unique())
print(list_Liga)

['Juniorinnen B' 'Juniorinnen C' 'Junioren B 1. Stärkeklasse'
 'Junioren C 1. Stärkeklasse' 'Junioren A Promotion'
 'Junioren C 2. Stärkeklasse' 'Junioren B 2. Stärkeklasse'
 'Junioren B Promotion' 'Junioren C Promotion' 'Youth League C'
 'Frauen 4. Liga' 'U-16 (Frauen)' 'Frauen 3. Liga' '4. Liga'
 'Youth League B' 'U-15' 'Junioren A' '5. Liga' '3. Liga' 'Frauen 2. Liga'
 'U-18 (Frauen)' "U-18 Women's Cup" 'Nationalliga B (Frauen)']


In [33]:
mask2 = df_spiel["Person-Nr anonymisiert"].isin(df_liz["Person-Nr anonymisiert"]).copy()
df_games_all = df_spiel[mask2].copy()

df_games_all = pd.merge(df_liz[["Person-Nr anonymisiert", "Geschlecht Person"]], df_games_all, on="Person-Nr anonymisiert", how="right")


df_games_all["Anz. Tore"] = (
    pd.to_numeric(df_games_all["Anz. Tore"], errors="coerce")
      .fillna(0)
      .astype(int)
)


# list of best players
all_players = (
    df_games_all.groupby("Person-Nr anonymisiert", as_index=False).agg({
        "Anz. Tore": "sum",
        "Total Einsatz-Minuten": "sum",
        "Liga": lambda x: ", ".join(sorted(set(x.dropna()))),
        "Position": lambda x: ", ".join(sorted(set(x.dropna()))), 
        "Vereinsname": "last", 
        "Geschlecht Person": "first"
        })
        .rename(columns={
            "Anz. Tore": "Goals", "Person-Nr anonymisiert" : "Id-Player", 
            "Total Einsatz-Minuten":"Minutes", "Geschlecht Person":"Sex"})
        .sort_values("Goals", ascending=False)
)
all_players.info()

all_players.drop(columns=["Goals", "Minutes", "Position","Sex"], inplace=True)
all_players.head()
all_players.info()
print(all_players.columns.to_list())
all_players.to_csv("../data/all_players.csv", index=False, encoding="utf-8") 



<class 'pandas.core.frame.DataFrame'>
Index: 20566 entries, 13672 to 20544
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id-Player    20566 non-null  int64 
 1   Goals        20566 non-null  int64 
 2   Minutes      20566 non-null  int64 
 3   Liga         20566 non-null  object
 4   Position     20566 non-null  object
 5   Vereinsname  20566 non-null  object
 6   Sex          20566 non-null  object
dtypes: int64(3), object(4)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 20566 entries, 13672 to 20544
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id-Player    20566 non-null  int64 
 1   Liga         20566 non-null  object
 2   Vereinsname  20566 non-null  object
dtypes: int64(1), object(2)
memory usage: 642.7+ KB
['Id-Player', 'Liga', 'Vereinsname']
